In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

# TensorFlow and tf.keras
import tensorflow as tf

import os
from tensorflow.keras import layers

from tensorflow import keras

# Helper libraries
import imageio
import numpy as np
import time
import matplotlib.pyplot as plt
from IPython import display
import PIL
import glob
from scipy import signal
import os 

import tensorflow as tf
import matplotlib.pyplot as plt
import scipy.io
from scipy import stats
from sklearn.preprocessing import OneHotEncoder
from sklearn.utils import resample
from sklearn.utils import class_weight
from sklearn.model_selection import KFold
import time
import os
from scipy.fftpack import fft,ifft

import sys
import json
import shutil
import argparse
import matplotlib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as patches

from sklearn import svm
from scipy.signal import find_peaks
from scipy.signal import argrelmin,argrelmax
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
mm = MinMaxScaler()

from collections import Counter
import pickle

In [ ]:
import pickle
# load PPG data
file_open=open('xxx/UBFC_data/cycle_ppg_UBFC_T1_S1_S56.pl','rb')
user_data_x,user_data_y=pickle.load(file_open)
file_open.close()

# load rPPG data
file_open=open('xxx/UBFC_data/cycle_rppg_UBFC_T1_S1_S56.pl','rb')
user_data_rppgx,user_data_rppgy=pickle.load(file_open)
file_open.close()

y_list = list(Counter(user_data_rppgy).keys()) 

## WGAN

In [ ]:
checkpoint_dir_folder = 'xxx/UBFC_data/WGAN_Check_Point'

In [ ]:
#train_x == rppg
#except_X == ppg
#train_y == label


arr_size = 60
train_x =[]
except_x = []
train_y = []
keys_list = Counter(user_data_rppgy).keys()
for number_ in range(1,57):
  if number_ in keys_list:
    test_ID = number_
    user_test_data_arr = []
    cycle_rppg_arr_x = []
    cycle_rppg_arr_y = [] 
    for index, data in enumerate(user_data_rppgx) :
      if user_data_rppgy[index]!=test_ID:
        continue
      resample = scipy.signal.resample(data,arr_size)
      resample_nor  =  mm.fit_transform(resample.reshape(-1,1)).flatten()
      user_test_data_arr.append(resample_nor)
    mean_rppg_arr = np.mean(user_test_data_arr, axis=0) 
    for i in user_test_data_arr:
      if stats.pearsonr(i,mean_rppg_arr)[0]<0.9:
        continue
      else:
        cycle_rppg_arr_x.append(i)
        cycle_rppg_arr_y.append(test_ID)


    user_test_data_arr = []
    cycle_ppg_arr_x = []
    cycle_ppg_arr_y = [] 
    for index, data in enumerate(user_data_x) :
      if user_data_y[index]!=test_ID:
        continue
      resample = scipy.signal.resample(data,arr_size)
      resample_nor  =  mm.fit_transform(resample.reshape(-1,1)).flatten()
      user_test_data_arr.append(resample_nor)
    mean_arr = np.mean(user_test_data_arr, axis=0) 
    for i in user_test_data_arr:
      if stats.pearsonr(i,mean_arr)[0]<0.9:
        continue
      else:
        cycle_ppg_arr_x.append(i)
        cycle_ppg_arr_y.append(test_ID)

    mean_sub_data = []
    sub_data = []
    mean_arr = np.mean(cycle_ppg_arr_x, axis=0) 


    for index, i in enumerate(cycle_rppg_arr_x) :

      train_x.append(i)
      except_x.append(mean_arr)
      train_y.append(test_ID)

## Train WGAN

In [ ]:
for round_num in range(1,57):
    print(round_num)

    test_ID=round_num
    train_x_id =[]
    except_x_id = []
    train_y_id = []
    rppg_train_x_test_id=[]
    for index, i in enumerate(train_x) :
      if train_y[index]==test_ID:
        rppg_train_x_test_id.append(i)
        continue
      train_x_id.append(i)
      except_x_id.append(except_x[index])
      train_y_id.append(train_y[index])

    train_x_id=np.array(train_x_id)
    train_x_id = np.expand_dims(train_x_id,2)
    train_x_id.shape


    except_x_id=np.array(except_x_id)
    except_x_id = np.expand_dims(np.array(except_x_id),2)
    except_x_id.shape


    train_x_id = tf.convert_to_tensor(train_x_id,dtype=tf.float32)

    except_x_id = tf.convert_to_tensor(except_x_id,dtype=tf.float32)

    BATCH_SIZE = 100
    inputLength = 60

    train_dataset = tf.data.Dataset.from_tensor_slices(train_x_id).batch(BATCH_SIZE)
    except_dataset = tf.data.Dataset.from_tensor_slices(except_x_id).batch(BATCH_SIZE)


    InputLength = 60
    Batch = 100
    """######Generator"""

    class Generator(tf.keras.Model):
      def __init__(self, InputLength=InputLength, Batch=Batch):
        super(Generator, self).__init__()
        self.start = layers.Dense(InputLength,input_shape=(InputLength,1),dtype='float32')
        self.active = layers.LeakyReLU(alpha=0.2)
        self.cov = layers.Conv1D(filters=32,kernel_size = 3, padding = 'causal')
        self.out = layers.Conv1D(filters = 1,kernel_size = 5, activation='tanh',padding = 'causal')
        self.dense = layers.Dense(1,dtype='float32')
        self.inference_net = tf.keras.Sequential(
          [
          self.active,
          self.cov,
          self.active,
          self.cov,
          self.active,
          self.cov,
          self.active,
          self.out

          ]
        )
      def call(self, inputs):
        x = self.start(inputs)
        x = self.inference_net(x)
        return x
    generator = Generator()

    """######Discriminator"""

    class Discriminator(tf.keras.Model):
      def __init__(self, InputLength=InputLength, Batch=Batch):
        super(Discriminator, self).__init__()
        self.start = layers.Conv1D(filters = 32,kernel_size = 3, padding = 'causal',input_shape=(InputLength,1))
        self.active = layers.LeakyReLU(alpha=0.2)
        self.cov = layers.Conv1D(filters=32, kernel_size = 3, padding = 'causal')
        self.pool = layers.MaxPooling1D(pool_size=2)
        self.flaten = layers.Flatten(input_shape=(InputLength,1))
        self.dense = layers.Dense(64,dtype='float32')
        self.drop = layers.Dropout(0.4)
        self.out = layers.Dense(1, activation='tanh',dtype='float32')
        self.inference_net = tf.keras.Sequential(
          [
          self.start,
          self.active,
          self.pool,
          self.cov,
          self.active,
          self.pool,
          self.flaten,
          self.dense,
          self.drop,
          self.active,
          self.out,
          ]
        )
      def call(self, inputs):
        x = self.inference_net(inputs)
        return x
    discriminator = Discriminator()


    # Define mix functions
    def mixSignal(real,fake):
      seed = np.random.randint(0,InputLength/2,1)[0]
      mix = tf.concat((real[:,seed:int(InputLength/2+seed),:],fake[:,seed:int(InputLength/2+seed),:]),1)
      return mix
    def wasserstein_loss(y_true, y_pred):
      return tf.reduce_mean(y_true * y_pred)
    def gradient_penalty_loss(mix):

      """
      Computes gradient penalty based on prediction and weighted real / fake samples
      """
      with tf.GradientTape() as g:
        g.watch(mix)
        y = discriminator(mix)
      gradients = g.gradient(y, mix) 
      # compute the euclidean norm by squaring ...
      gradients_sqr = tf.math.pow(gradients,2*tf.ones_like(gradients))
      #   ... summing over the rows ...
      gradients_sqr_sum = tf.reduce_sum(gradients_sqr,axis=np.arange(1, len(gradients_sqr.shape)))
      #   ... and sqrt
      gradient_l2_norm = tf.math.sqrt(gradients_sqr_sum)  
      # compute lambda * (1 - ||grad||)^2 still for each single sample
      gradient_penalty = tf.math.pow((1 - gradient_l2_norm),2*tf.ones_like(gradient_l2_norm))
      # return the mean as loss over all the batch samples
      return tf.reduce_mean(gradient_penalty)


    def discriminator_loss(real_output, fake_output, mix):
        real_loss = wasserstein_loss(-0.5*tf.ones_like(real_output), real_output)
        fake_loss = wasserstein_loss(0.5*tf.ones_like(fake_output), fake_output)
        mix_loss = gradient_penalty_loss(mix)
        total_loss = real_loss+fake_loss+10*mix_loss
        return total_loss

    def generator_loss(fake_output,mix_output):
        mix_loss = wasserstein_loss(-0.5*tf.ones_like(fake_output), fake_output)
        fake_loss = wasserstein_loss(-0.5*tf.ones_like(fake_output), fake_output)
        return fake_loss+mix_loss

    generator_optimizer = tf.keras.optimizers.Adam(lr=0.00001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
    discriminator_optimizer = tf.keras.optimizers.Adam(lr=0.00001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)

    checkpoint_dir = 'xxx/UBFC_data/WAGN_training_checkpoints/training_checkpoints_'+str(test_ID)
    checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
    checkpoint = tf.train.Checkpoint(generator_optimizer=generator_optimizer,
                                    discriminator_optimizer=discriminator_optimizer,
                                    generator=generator,
                                    discriminator=discriminator)


    """######training"""


    @tf.function
    def train_step(train_x, except_x):
      for i in range(5):
        noise = tf.random.normal([Batch, InputLength,1])
        with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
          generated_Data = generator(train_x, training=True)
          mix_Data = mixSignal(except_x,generated_Data)

          real_output = discriminator(except_x, training=True)
          fake_output = discriminator(generated_Data, training=True)
          mix_output = discriminator(mix_Data, training=True)

          gen_loss = generator_loss(fake_output,mix_output)
          disc_loss = discriminator_loss(real_output, fake_output,mix_Data)

        gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
        gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)
        if i==4:
          generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
        discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))
      return gen_loss,disc_loss


    def train(train_x_dataset,except_x_dataset, epochs):
      for epoch in range(epochs):
        start = time.time()
        num = 0
        for image_batch, image_batch_y in zip(train_x_dataset,except_x_dataset ):
          gen_loss,disc_loss = train_step(image_batch,image_batch_y)
          num = num+1
          if num%500==0:
            print ('generator loss {} discriminator loss {} sec'.format(gen_loss, disc_loss))

            display.clear_output(wait=True)

        if (epoch + 1) % 15 == 0:
          checkpoint.save(file_prefix = checkpoint_prefix)

          print ('Time for epoch {} is {} sec'.format(epoch + 1, time.time()-start))
          print ('generator loss {} discriminator loss {} sec'.format(gen_loss, disc_loss))

      display.clear_output(wait=True)
      
    def generate_and_save_images(model, epoch, test_input):
        predictions = model(test_input, training=False)
        for i in range(3):
            plt.plot(predictions[i].numpy()) 

        plt.show()


    train(train_dataset, except_dataset, 500)

    # get pre result
    arr_size = 60
    user_test_data_arr = []
    cycle_ppg_arr_x = []
    cycle_ppg_arr_y = [] 
    train_data_x = []
    test_data_x = []
    for index, data in enumerate(user_data_x) :
      if user_data_y[index]!=test_ID:
        continue
      resample = scipy.signal.resample(data,arr_size)
      resample_nor  =  mm.fit_transform(resample.reshape(-1,1)).flatten()
      user_test_data_arr.append(resample_nor)
    mean_arr = np.mean(user_test_data_arr, axis=0) 
    for i in user_test_data_arr:
      if stats.pearsonr(i,mean_arr)[0]<0.9:
        continue
      else:
        cycle_ppg_arr_x.append(i)
        cycle_ppg_arr_y.append(test_ID)
        train_data_x.append(i)

    for index, data in enumerate(user_data_x) :

      if user_data_y[index]==test_ID:
        continue
      resample = scipy.signal.resample(data, arr_size)
      resample_nor  = mm.fit_transform(resample.reshape(-1,1)).flatten()
      cycle_ppg_arr_x.append(resample_nor)
      cycle_ppg_arr_y.append(user_data_y[index])
      test_data_x.append(resample_nor)
      

    rppg_train_x_test_id = np.expand_dims(rppg_train_x_test_id,2)
    preds = generator(rppg_train_x_test_id,training=False)

    b = np.squeeze(rppg_train_x_test_id)
    preds_b = np.squeeze(preds)



    file_open=open('wgan_rppg_result_'+str(test_ID)+'.pl','wb')
    pickle.dump((b,preds_b),file_open)
    file_open.close()

## GP

In [ ]:
!pip install gpflow

In [ ]:
from typing import Tuple, Optional
import tempfile
import pathlib

import datetime
import io
import matplotlib.pyplot as plt

import numpy as np
import tensorflow as tf
import gpflow

from gpflow.config import default_float
from gpflow.ci_utils import ci_niter
from gpflow.utilities import to_default_float

import warnings

warnings.filterwarnings("ignore")

In [ ]:
for number_ in (y_list):

    test_ID=number_
    train_x_id =[]
    except_x_id = []
    train_y_id = []
    rppg_train_x_test_id=[]
    for index, i in enumerate(train_x) :
      if train_y[index]==test_ID and test_ID!=10:
        rppg_train_x_test_id.append(i)
        continue
      train_x_id.append(i)
      except_x_id.append(except_x[index])
      train_y_id.append(train_y[index])


    train_x_id=np.array(train_x_id)
    train_x_id = np.expand_dims(train_x_id,2)
    train_x_id.shape


    except_x_id=np.array(except_x_id)
    except_x_id = np.expand_dims(np.array(except_x_id),2)
    except_x_id.shape


    train_x_id = tf.convert_to_tensor(train_x_id,dtype=tf.float64)
    except_x_id = tf.convert_to_tensor(except_x_id,dtype=tf.float64)

    BATCH_SIZE = 32
    inputLength = 60

    train_dataset = tf.data.Dataset.from_tensor_slices(train_x_id).batch(BATCH_SIZE)
    except_dataset = tf.data.Dataset.from_tensor_slices(except_x_id).batch(BATCH_SIZE)

    # kernel = gpflow.kernels.SquaredExponential(variance=2.0)
    # kernel = gpflow.kernels.Linear()
    # kernel = gpflow.kernels.RationalQuadratic()
    kernel = gpflow.kernels.Matern32()


    likelihood = gpflow.likelihoods.Gaussian()
    inducing_variable = np.linspace(0, 60, 60).reshape(-1, 1)

    model = gpflow.models.SVGP(
    kernel=kernel, likelihood=likelihood, inducing_variable=inducing_variable
    )

    optimizer = tf.optimizers.Adam()
    def optimization_step(model: gpflow.models.SVGP, batch: Tuple[tf.Tensor, tf.Tensor]):
        with tf.GradientTape(watch_accessed_variables=False) as tape:
            tape.watch(model.trainable_variables)
            loss = model.training_loss(batch)
        grads = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(grads, model.trainable_variables))
        return loss


    def simple_training_loop(model: gpflow.models.SVGP, epochs: int = 1, logging_epoch_freq: int = 10):
        tf_optimization_step = tf.function(optimization_step)

        batches = iter(train_dataset)
        for epoch in range(epochs):
            for _ in range(ci_niter(num_batches_per_epoch)):
                tf_optimization_step(model, next(batches))

            epoch_id = epoch + 1
            if epoch_id % logging_epoch_freq == 0:
                tf.print(f"Epoch {epoch_id}: ELBO (train) {model.elbo(data)}")


    data = (train_x_id, except_x_id)

    
    train_dataset = tf.data.Dataset.from_tensor_slices((train_x_id, except_x_id))
    test_dataset = tf.data.Dataset.from_tensor_slices((train_x_id, except_x_id))

    batch_size = 32
    num_features = 60
    prefetch_size = tf.data.experimental.AUTOTUNE
    shuffle_buffer_size = len(train_x_id) // 2
    num_batches_per_epoch =  len(train_x_id) // batch_size

    original_train_dataset = train_dataset
    train_dataset = (
        train_dataset.repeat()
        .prefetch(prefetch_size)
        .shuffle(buffer_size=shuffle_buffer_size)
        .batch(batch_size)
    )

    simple_training_loop(model, epochs=10, logging_epoch_freq=5)



    test_x_id=np.array(rppg_train_x_test_id)
    test_x_id = np.expand_dims(np.array(test_x_id),2)
    test_x_id.shape
    test_x_id = tf.convert_to_tensor(test_x_id,dtype=tf.float64)

    model.predict_f_compiled = tf.function(
        model.predict_f, input_signature=[tf.TensorSpec(shape=[None, 60,1], dtype=tf.float64)]
    )

    samples_input = tf.convert_to_tensor(test_x_id, dtype=default_float())
    original_result = model.predict_f_compiled(samples_input)
    test_result = original_result[0].numpy()
    plt.plot(original_result[0][0])

    file_open=open('xxx/UBFC_data/wgan_rppg_result_T1_GP_'+str(test_ID)+'.pl','wb')
    pickle.dump((test_result,1),file_open)
    file_open.close()


    tf.saved_model.save(model, 'xxx/UBFC_data/GP_model/model'+str(test_ID))

## GMM

In [ ]:
!pip install nnmnkwii
import sklearn
from nnmnkwii.baseline.gmm import MLPG

In [ ]:
from tqdm import tqdm
for round_num in tqdm(y_list+[10]) :
    
    test_ID=round_num
    train_x_id =[]
    except_x_id = []
    train_y_id = []
    for index, i in enumerate(train_x) :
      if train_y[index]==test_ID:
        continue
      train_x_id.append(i)
      except_x_id.append(except_x[index])
      train_y_id.append(train_y[index])

    train_x_id=np.array(train_x_id)
    train_x_id.shape
    except_x_id=np.array(except_x_id)

    XY = np.concatenate((train_x_id, except_x_id), axis=-1)
    gmm = sklearn.mixture.GaussianMixture(n_components=3)
    _ = gmm.fit(XY)
    paramgen = MLPG(gmm,  windows=[(0,0, np.array([1.0]))], swap = True)

    file_open=open('xxx/UBFC_data/wgan_rppg_result_GMM_'+str(number_)+'.pl','rb')
    rppg_train_x_test_id,_=pickle.load(file_open)
    file_open.close()


    try:
      generated = paramgen.transform(rppg_train_x_test_id)
    except:
      print('except',test_ID)
      continue

    file_open=open('xxx/GMM_rppg_result_T1_'+str(test_ID)+'.pl','wb')
    pickle.dump((rppg_train_x_test_id,generated),file_open)
    file_open.close()

